<a href="https://colab.research.google.com/github/Shreejan-git/pytorch-complete-course/blob/main/pytorch_nn_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

In [2]:
print(torch.__version__)

2.0.1+cu118


In [3]:
!nvidia-smi

Thu Sep  7 08:01:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Notes while creating a custom model:**


1.   If we are creating our custom neural network module, it must inherit from nn.Module. https://pytorch.org/docs/stable/generated/torch.nn.Module.html
2.   Always call the __init__ (constructor) of the parent class using the super().__init__(). This is a mandatory step. If we do not call the parents's constructor, we will have problems while training and saving the model.
3. When we are inheriting from nn.Module, it is mandatory to add forward method.
4. Below, nn.parameter's work is to mark that variable as a learnable parameter. nn.parameter will track it while training.


In [15]:
class SimpleLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.weight = nn.Parameter(torch.rand(1,
                               requires_grad=True),
                               dtype=torch.float32)
    self.bias = nn.Parameter(torch.rand(1,
                             requires_grad=True,
                             dtype=torch.float32))

  def forward(self, X: torch.Tensor) -> torch.Tensor:
    return self.weight * X + self.bias

**Resources**

Pytorch Loss function: https://pytorch.org/docs/stable/nn.html#loss-functions , https://neptune.ai/blog/pytorch-loss-functions

Pytorch optimization functions: https://pytorch.org/docs/stable/optim.html

In [16]:
# defining the loss function. L1Loss is MAE loss function.
loss_fun = nn.L1Loss()
print(loss_fun)

L1Loss()


In [ ]:
# defining the optimization function.
# We can define any optimization function. like Adam, SGD, Adagrad
# 1st parameter of SGD is the model's parameters. We can get this by writing model.parameters().
#Give the exact name of the variable which we named while initializing the Class
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr = 0.001,)